In [2]:
import os
import time
from dotenv import load_dotenv
load_dotenv()

pinecone_api_key = os.environ.get('PINECONE_API_KEY')
openai_api_key = os.environ.get('OPENAI_API_KEY')

In [4]:
# 기반 데이터셋 로드
from datasets import load_dataset

dataset = load_dataset("lcw99/wikipedia-korean-20221001", split='train[:100]')

In [5]:
# 라마인덱스 네이티브 형태의 다큐먼트로 파싱하기 위해 데이터프레임으로 정의
data = dataset.to_pandas()[['id', 'text', 'title']]
data.drop_duplicates(subset='text', keep='first', inplace=True)
data.head()

,id,text,title
0,5,"제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대...",지미 카터
1,9,"수학(數學, , 줄여서 math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루...",수학
2,10,"수학에서 상수란 그 값이 변하지 않는 불변량으로, 변수의 반대말이다. 물리 상수와는...",수학 상수
3,19,"문학(文學, )은 언어를 예술적 표현의 제재로 삼아 새로운 의미를 창출하여, 인간과...",문학
4,20,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,나라 목록


In [6]:
#라마인덱스 다큐먼트 오브젝트로서 텍스트 데이터를 메타와 함께 정의
from llama_index.core import Document

docs = []

for i, row in data.iterrows():
    docs.append(Document(
        text=row['text'],
        doc_id=row['id'],
        extra_info={'title': row['title']}
    ))

In [8]:
# # Document 확인
# docs[0]

In [9]:
# 노드로 청킹
# Node는 llamaindex의 기본 문서 유닛
from llama_index.core.node_parser import SimpleNodeParser

parser = SimpleNodeParser()

nodes = parser.get_nodes_from_documents(docs)
nodes[0]

TextNode(id_='ab3ac2b3-a2ac-44c1-92d4-1a015f7ca13f', embedding=None, metadata={'title': '지미 카터'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'title': '지미 카터'}, hash='6c4f6250b2c71196e0876bb59e94d9496a6fae62ee2104a452c19434f7e2bc0d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='d392f6b4-eda4-4cf5-82a7-14218e93e317', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='b1c0fcd751e6cd8d42a1d9900d9c841da7c7ff46b5583259d4b56955547c75f6')}, text='제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n\n정계 입문 \n1962년 조지아주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 

In [10]:
# 파인콘 연결
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=pinecone_api_key)

In [11]:
from time import sleep

index_name="quickstart"

if index_name in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.delete_index(index_name)

pc.create_index(
   name=index_name,
   dimension=1536,
   metric="dotproduct",
   spec=ServerlessSpec(
       cloud='aws',
       region='us-east-1'
   )
)

while not pc.describe_index(index_name).status['ready']:
    sleep(1)
index = pc.Index(index_name)
sleep(1)
index_stats = index.describe_index_stats()
print(index_stats)

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [12]:
# 라마인덱스 벡터스토어로 정의
from llama_index.vector_stores.pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(pinecone_index=index)

In [15]:
# embeddings upsert
from llama_index.core import VectorStoreIndex, StorageContext, ServiceContext
from llama_index.embeddings.openai import OpenAIEmbedding

# 벡터스토어 인덱스에 들어가는 스토리지 컴포넌트 정의
# 참조할 storage 정의
storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)

# 서비스 컨텍스트에 해당 인덱스를 활용한 연결 컴포넌트 정의(임베딩 모델 +@)
embed_model = OpenAIEmbedding(model='text-embedding-ada-002', embed_batch_size=100) # embed_batch_size는 한번에 embed할 node의 수
service_context = ServiceContext.from_defaults(embed_model=embed_model)

index = VectorStoreIndex.from_documents(
    docs, storage_context=storage_context,
    service_context=service_context
)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_21420\1469514084.py:13: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embed_model)


Upserted vectors:   0%|          | 0/516 [00:00<?, ?it/s]

In [16]:
query_engine = index.as_query_engine()
res = query_engine.query("지미 카터가 누구야?")
print(res)

지미 카터는 민주당 출신으로 미국의 39대 대통령이다. 그는 미국의 역사에서 중요한 역할을 한 인물로, 대통령으로서 다양한 정책과 외교 노력을 펼쳤으며, 후에는 인권과 중재 역할에 대한 공로로 노벨 평화상을 수상한 인물이다.


llamaindex는 langchain에 비해 간단하게 사용가능 -> high level framework